In [ ]:
from PIL import Image
import numpy as np
import pandas as pd
import csv
import os
from skimage.feature import graycomatrix, graycoprops
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, matthews_corrcoef, cohen_kappa_score
from sklearn.svm import SVC

In [ ]:
inputDirectory ="/content/pet_dataset.zip"

In [ ]:
with open("/content/pet_dataset.zip", 'a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Image','red_sum', 'green_sum', 'blue_sum', 'correlation', 'energy', 'contrast', 'homogeneity','label'])

In [ ]:
def analyze_image(image_path):
    # Load the image
    image = Image.open(image_path)
    # Convert the image to RGB mode (if it's not already)
    image = image.convert("RGB")
    # Convert the image to a numpy array
    image_array = np.array(image)

    # Calculate the sums of colors on each channel
    red_sum = np.sum(image_array[:, :, 0])
    green_sum = np.sum(image_array[:, :, 1])
    blue_sum = np.sum(image_array[:, :, 2])

    # Convert the image to grayscale
    grayscale_image = image.convert("L")
    # Convert the grayscale image to a numpy array
    grayscale_array = np.array(grayscale_image)

# Calculate the texture properties using greycomatrix and greycoprops
    glcm = graycomatrix(grayscale_array, distances=[1], angles=[0], symmetric=True, normed=True)
    correlation = graycoprops(glcm, 'correlation')[0, 0]
    energy = graycoprops(glcm, 'energy')[0, 0]
    contrast = graycoprops(glcm, 'contrast')[0, 0]
    homogeneity = graycoprops(glcm, 'homogeneity')[0, 0]

    # Return the calculated features as a list
    features = [image_array, red_sum, green_sum, blue_sum, correlation, energy, contrast, homogeneity]
    return features

In [ ]:
for folder in os.listdir(inputDirectory):
    print("Inside folder {}".format(folder))
    folderPath = os.path.join(inputDirectory, folder)
    for img in os.listdir(folderPath):
        imgPath = os.path.join(folderPath, img)
        features = analyze_image(imgPath)
        features.append(folder)
        with open("/content/pet_dataset.zip", 'a', newline='') as file:
            writer = csv.writer(file)
            writer.writerow(features)
        print("Processed Image: {}".format(imgPath))

In [ ]:
df = pd.read_csv("/content/pet_dataset.zip")

In [ ]:
df.head()

In [ ]:
df = df.drop(columns = ['Image'])


In [ ]:
xTrain, xTest, yTrain, yTest = train_test_split(df.drop(columns=['label']), df['label'], train_size=0.8, random_state=42)

In [ ]:
labelEncoder = LabelEncoder()
labelEncoder.fit(yTrain)
print(labelEncoder.classes_)
yTrain = labelEncoder.transform(yTrain)
yTest = labelEncoder.transform(yTest)

In [ ]:
scaler = StandardScaler()
scaler.fit(xTrain)
xTrain = scaler.transform(xTrain)
xTest = scaler.transform(xTest)

In [ ]:
with open("/content/pet_dataset.zip"), 'a', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Model','Accuracy','Precision','Senstivity','F1 Score','MCC Score','Kappa Coeff'])

In [ ]:
modelSVC = SVC()

In [ ]:
modelSVC.fit(xTrain, yTrain)